In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
niter = 1000
optimise_mu = [False, False, True]
exact = [False, True, True]
mc_samples = 100
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             optimise_mu = optimise_mu[i], exact = exact[i],
                             mc_samples = mc_samples, kernel = kernel,
                             niter = niter)
            futures += [f]

In [5]:
# Check on futures
#futures

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,n,d,noise,kernel))